In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5bf4dea66fd54d63a8953e0041953a10 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=76c5dfcc47624d0c94845b7301058ddb in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [ ]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

# create datasets

In [ ]:
# third party
import numpy as np

In [ ]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

In [ ]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

In [ ]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [ ]:
@sy.syft_function()
def square_number(data: float) -> float:
    print("executing sub function")
    return data**2


@sy.syft_function()
def aggregate(data: list[float]) -> float:
    print("Aggregating results...")
    return sum(data) / len(data)


client_low_ds.code.submit(square_number)
client_low_ds.code.submit(aggregate)

In [ ]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    print("executing main function")

    results = []
    for item in data:
        job = domain.launch_job(square_number, data=item)
        results.append(job.result)
    job = domain.launch_job(aggregate, data=results)
    return job.result


# TODO
# res = compute_mean(data=data_low.mock, blocking=False)

In [ ]:
client_low_ds.code.request_code_execution(compute_mean)

In [ ]:
client_low_ds.code.compute_mean(data=data_low)

## Data Owner: Sync to high side

In [ ]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [ ]:
low_state

In [ ]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [ ]:
diff_state = compare_states(low_state, high_state)

In [ ]:
diff_state

In [ ]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

In [ ]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

In [ ]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [ ]:
client_low.apply_state(resolved_state_low)

In [ ]:
client_high.apply_state(resolved_state_high)

In [ ]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [ ]:
data_high = client_high.datasets[0].assets[0]

In [ ]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

In [ ]:
# wait for the result
job_high.wait().get()

In [ ]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [ ]:
job_info

In [ ]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [ ]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

In [ ]:
# Need to refresh Job because we overwrite result
job_high = client_high.code.compute_mean.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [ ]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [ ]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

if isinstance(low_state, sy.SyftError):
    print("low")
    print(low_state.message)

if isinstance(high_state, sy.SyftError):
    print("high")
    print(high_state.message)

In [ ]:
diff_state_2 = compare_states(low_state, high_state)

In [ ]:
resolved_state_low, resolved_state_high = resolve(diff_state_2)

In [ ]:
print(resolved_state_low)
print()
print(resolved_state_high)

In [ ]:
res = client_low.apply_state(resolved_state_low)
res

In [ ]:
client_high.apply_state(resolved_state_high)

In [59]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [60]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

# Run code low

## Run

In [62]:
client_low_ds._fetch_api(client_low.credentials)

In [67]:
client_low_ds.code.compute_mean.jobs[0].result.get()

17.0

In [68]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [69]:
res_low.id == job_high.result.id

False

In [70]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

AssertionError: 

In [71]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
17.0

In [72]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

AssertionError: 

In [73]:
private_high.mean()

17.0

In [74]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = d76d16a7bf3341deb0caccd402bb4fd5
    status: completed
    has_parent: False
    result: ActionDataEmpty <None>
    logs:

0 Log 8ad86aa308364ea2a98ce8cb4341093f not available
JOB COMPLETED
    
```

In [75]:
job_low.wait().get()

17.0

In [76]:
job_low.logs()

Log 8ad86aa308364ea2a98ce8cb4341093f not available


In [77]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)